In [ ]:
import os
import math

import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
import sklearn.gaussian_process as gp
from mpl_toolkits.mplot3d import Axes3D

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
import commonmodules as cm

In [ ]:
filename = "HeCS.xlsx"
df = pd.read_excel(filename, sheet_name="dv1")
#print(df.head())
#for cname in df.columns:
#    print(cname, df[cname].dtype)
x = df[['v', 'T']].values
y = df['RC'].values
X = np.array(x)
Y = np.array(y)
temp_values = np.unique(X[:, 1])
vib_values = np.unique(X[:, 0])

xdim = len(temp_values)
ydim = len(vib_values)

Xp = np.zeros((xdim, ydim), dtype=float)
Yp = np.zeros((xdim, ydim), dtype=float)
Zp = np.zeros((xdim, ydim), dtype=float)
for xidx in range(xdim):
    t = temp_values[xidx]
    for yidx in range(ydim):
        v =  vib_values[yidx]
        Xp[xidx, yidx] = float(t)
        Yp[xidx, yidx] = float(v)
        Zp[xidx, yidx] = df[(df['v'] == v) & (df['T'] == t)]['RC'].values[0]

#fig = plt.figure(figsize=(10,8))
fig = plt.figure(figsize=plt.figaspect(2.))
plt.gcf().set_size_inches(40, 30)
ax = fig.add_subplot(2,1,1, projection='3d')
surf = ax.plot_surface(Xp, Yp, Zp, rstride=1, cstride=1, cmap='jet', linewidth=0, antialiased=False)
plt.show()

plt.clf()
fig = plt.figure()
ax = fig.add_subplot(projection='3d')

for xidx in range(xdim):
    t = temp_values[xidx]
    for yidx in range(ydim):
        v =  vib_values[yidx]
        x = float(t)
        y = float(v)
        z = df[(df['v'] == v) & (df['T'] == t)]['RC'].values[0]
        ax.scatter(x, y, z, marker="o", color="b")

ax.set_xlabel('X Label')
ax.set_ylabel('Y Label')
ax.set_zlabel('Z Label')
plt.gcf().set_size_inches(20, 15)

plt.show()

In [ ]:
print("X shape :", X.shape)
print("Y shape :", Y.shape)
vib_values_uniq = np.unique(X[:, 0])
print("Vibration values:", vib_values_uniq)
vibtest = vib_values_uniq[int(len(vib_values_uniq)/2)]
print("Testing with vibration value:", vibtest)
train_xy = []
train_z = []
test_xy = []
test_z = []
for i in range(X.shape[0]):
    if X[i][0] == vibtest:
        test_xy.append(X[i])
        test_z.append(Y[i])
    else:
        train_xy.append(X[i])
        train_z.append(Y[i])
train_xy = np.array(train_xy)
train_z = np.array(train_z)
test_xy = np.array(test_xy)
test_z = np.array(test_z)
print("Train X shape:", train_xy.shape)
print("Train Y shape:", train_z.shape)
print("Test X shape:", test_xy.shape)
print("Test Y shape:", test_z.shape)        

In [ ]:
import sklearn.preprocessing as skp
# log scale for Y
train_z = np.log(train_z)
test_z = np.log(test_z)
# scale values to [0, 1]
scaler = skp.MinMaxScaler()
train_xy = scaler.fit_transform(train_xy)
test_xy = scaler.transform(test_xy)
# same for z
train_z = scaler.fit_transform(train_z.reshape(-1, 1))
test_z = scaler.transform(test_z.reshape(-1, 1))

In [ ]:
model = cm.build_model_GP_1 (train_xy, train_z)

z_pred, std = model.predict(train_xy, return_std=True)
trainmse = 0.0
cont = 0.0
for i in range(train_z.shape[0]):
    x = train_xy[i,0]
    y = train_xy[i,1]
    z = train_z[i]
    zpred = z_pred[i]
    zstd = std[i]
    
    trainmse += (zpred-z)**2
    cont += 1.0

    print("Train, %10.7f , %10.7f , %10.7f , %10.7f , %10.7f"%(z, y, z, zpred, zstd))

trainmse = trainmse/cont

z_pred, std = model.predict(test_xy, return_std=True)
mse = 0.0
cont = 0.0
for i in range(test_z.shape[0]):
    x = test_xy[i,0]
    y = test_xy[i,1]
    z = test_z[i]
    zpred = z_pred[i]
    zstd = std[i]
    
    mse += (zpred-z)**2
    cont += 1.0

    print("Test, %10.7f , %10.7f , %10.7f , %10.7f , %10.7f"%(z, y, z, zpred, zstd))

mse = mse/cont

print("MSE ", mse, " , TrainMSE ,", trainmse, flush=True)